In [1]:
import requests

# 目标网址
url = "https://www.icourse163.org/course/NHDX-1463126169"

# 设置请求头，模拟浏览器访问
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

# 发起请求
response = requests.get(url, headers=headers)

# 将响应的内容保存为HTML文件
file_path = 'icourse163_page.html'
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(response.text)

file_path


'icourse163_page.html'

In [4]:
!pip install selenium

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 1.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [selenium]5/6 [selenium]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.37.20 which is incompatible.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from selenium import webdriver
from time import sleep

# 使用 Selenium Manager 自动管理 ChromeDriver
driver = webdriver.Chrome()
# driver = webdriver.Firefox()

# 打开目标网页
driver.get("https://www.icourse163.org/course/NHDX-1463126169")

# 等待页面加载
sleep(5)

# 保存网页源代码
with open("icourse163_selenium_saved.html", "w", encoding="utf-8") as file:
    file.write(driver.page_source)
    

driver.quit()


In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 初始化 WebDriver，启动 Chrome 浏览器
driver = webdriver.Chrome()

# 打开目标网页
url = "https://www.icourse163.org/course/NHDX-1463126169"
driver.get(url)

# 设置浏览器窗口大小
driver.set_window_size(1082, 672)

# 等待“课程评价”按钮加载，并点击
try:
    # 等待按钮可点击
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "review-tag-button"))
    )
    review_button.click()  # 点击“课程评价”按钮
    print("点击了'课程评价'按钮")
except Exception as e:
    print(f"未能找到或点击按钮，错误: {e}")

# 保存当前页面的 HTML 源代码
time.sleep(5)  # 等待页面完全加载
file_name = "course_reviews.html"
with open(file_name, "w", encoding="utf-8") as file:
    file.write(driver.page_source)  # 将页面源码写入文件
print(f"页面已保存为 {file_name}")

# 关闭浏览器
driver.quit()
print("浏览器已关闭")


点击了'课程评价'按钮
页面已保存为 course_reviews.html
浏览器已关闭


In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 创建一个 Chrome 浏览器实例
driver = webdriver.Chrome()


# 用于存储所有评论数据的列表
all_reviews = []

# 打开指定的网页
url = "https://www.icourse163.org/course/NHDX-1463126169"
driver.get(url)

# 设置浏览器窗口大小
driver.set_window_size(1082, 672)

# 点击“课程评价”按钮，等待按钮可点击
try:
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "review-tag-button"))
    )
    review_button.click()  # 点击按钮
    time.sleep(3)  # 等待页面加载
except Exception as e:
    print(f"无法点击课程评价按钮: {e}")

# 定义一个函数用于提取页面上的评论内容
def extract_reviews():
    # 查找页面上的所有评论元素
    review_elements = driver.find_elements(By.XPATH, '//div[@class="ux-mooc-comment-course-comment_comment-list_item"]')
    
    # 遍历每个评论元素
    for review_element in review_elements:
        review_data = {}
        try:
            # 提取用户名称
            review_data['user'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_user-info_name'
            ).text

            # 提取评论时间
            review_data['time'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_comment-info_time'
            ).text

            # 提取评分（星星数）
            stars = review_element.find_elements(
                By.CLASS_NAME, 'ux-icon-custom-rating-favorite'
            )
            review_data['rating'] = len(stars)  # 星星数表示评分

            # 提取评论内容
            review_data['content'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_content'
            ).text
        except Exception as e:
            print(f"解析评论时出错: {e}")

        # 将每个评论数据添加到总列表中
        all_reviews.append(review_data)

# 定义一个函数用于点击下一页按钮
def next_page():
    try:
        # 查找并点击下一页按钮
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#comment-section > div > div.ux-mooc-comment-course-comment_pager > ul > li.ux-pager_btn.ux-pager_btn__next > a'))
        )
        next_button.click()  # 点击下一页按钮
        time.sleep(3)  # 等待页面加载
        return True  # 成功点击下一页按钮
    except Exception as e:
        print(f"无法点击下一页: {e}")
        return False  # 无法点击下一页按钮

# 提取指定页数的评论
num_pages = 10  # 设置要提取的页数
for page in range(1, num_pages + 1):
    print(f"正在提取第 {page} 页的评论...")
    extract_reviews()  # 提取当前页面的评论

    # 如果不是最后一页，尝试翻页
    if not next_page():
        print("已到达最后一页，停止翻页。")
        break

# 将所有提取的评论数据保存为 CSV 文件
df = pd.DataFrame(all_reviews)
output_file = "course_reviews1.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"数据已保存到 {output_file}")

# 关闭浏览器
driver.quit()
print("所有页面处理完毕，浏览器已关闭")


正在提取第 1 页的评论...
正在提取第 2 页的评论...
正在提取第 3 页的评论...
正在提取第 4 页的评论...
正在提取第 5 页的评论...
正在提取第 6 页的评论...
正在提取第 7 页的评论...
正在提取第 8 页的评论...
无法点击下一页: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=135.0.7049.115)
Stacktrace:
0   chromedriver                        0x00000001053bea54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x00000001053b6cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x0000000104f02864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000104edc6e8 chromedriver + 132840
4   chromedriver                        0x0000000104f71404 cxxbridge1$string$len + 546564
5   chromedriver                        0x0000000104f89f8c cxxbridge1$string$len + 647820
6   chromedriver                        0x0000000104f3d7ec cxxbridge1$string$len + 334572
7   chromedriver                        0x0000000105383ccc cxxbridge1$str$ptr + 2562928
8   chromedriv

如果你浏览器有问题 可以用其他浏览器代替
self.driver = webdriver.Firefox()
将 Chrome WebDriver 换成 Firefox WebDriver。
。

In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 创建一个 Firefox 浏览器实例
driver = webdriver.Firefox()  # 使用Firefox代替Chrome

# 用于存储所有评论数据的列表
all_reviews = []

# 打开指定的网页
url = "https://www.icourse163.org/course/NHDX-1463126169"
driver.get(url)

# 设置浏览器窗口大小
driver.set_window_size(1082, 672)

# 点击“课程评价”按钮，等待按钮可点击
try:
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "review-tag-button"))
    )
    review_button.click()  # 点击按钮
    time.sleep(3)  # 等待页面加载
except Exception as e:
    print(f"无法点击课程评价按钮: {e}")

# 定义一个函数用于提取页面上的评论内容
def extract_reviews():
    # 查找页面上的所有评论元素
    review_elements = driver.find_elements(By.XPATH, '//div[@class="ux-mooc-comment-course-comment_comment-list_item"]')
    
    # 遍历每个评论元素
    for review_element in review_elements:
        review_data = {}
        try:
            # 提取用户名称
            review_data['user'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_user-info_name'
            ).text

            # 提取评论时间
            review_data['time'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_comment-info_time'
            ).text

            # 提取评分（星星数）
            stars = review_element.find_elements(
                By.CLASS_NAME, 'ux-icon-custom-rating-favorite'
            )
            review_data['rating'] = len(stars)  # 星星数表示评分

            # 提取评论内容
            review_data['content'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_content'
            ).text
        except Exception as e:
            print(f"解析评论时出错: {e}")

        # 将每个评论数据添加到总列表中
        all_reviews.append(review_data)

# 定义一个函数用于点击下一页按钮
def next_page():
    try:
        # 查找并点击下一页按钮
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#comment-section > div > div.ux-mooc-comment-course-comment_pager > ul > li.ux-pager_btn.ux-pager_btn__next > a'))
        )
        next_button.click()  # 点击下一页按钮
        time.sleep(3)  # 等待页面加载
        return True  # 成功点击下一页按钮
    except Exception as e:
        print(f"无法点击下一页: {e}")
        return False  # 无法点击下一页按钮

# 提取指定页数的评论
num_pages = 5  # 设置要提取的页数
for page in range(1, num_pages + 1):
    print(f"正在提取第 {page} 页的评论...")
    extract_reviews()  # 提取当前页面的评论

    # 如果不是最后一页，尝试翻页
    if not next_page():
        print("已到达最后一页，停止翻页。")
        break

# 将所有提取的评论数据保存为 CSV 文件
df = pd.DataFrame(all_reviews)
output_file = "course_reviewsfox.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"数据已保存到 {output_file}")

# 关闭浏览器
driver.quit()
print("所有页面处理完毕，浏览器已关闭")


正在提取第 1 页的评论...
正在提取第 2 页的评论...
正在提取第 3 页的评论...
正在提取第 4 页的评论...
正在提取第 5 页的评论...
数据已保存到 course_reviewsfox.csv
所有页面处理完毕，浏览器已关闭


In [6]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # 访问网页
    driver.get("https://www.icourse163.org/course/NHDX-1463126169")
    driver.set_window_size(1728, 905)

    # 等待评论区加载完成
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "review-tag-num"))
    )

    # 点击评论区
    driver.find_element(By.ID, "review-tag-num").click()

    # 创建一个列表来存储所有评论
    all_comments = []

    # 定义一个函数来收集当前页面的评论
    def collect_comments_on_current_page():
        # 等待评论加载
        time.sleep(2)

        # 获取当前页面上的所有评论
        comments = driver.find_elements(By.CSS_SELECTOR, ".m-reviewitm")

        # 提取每条评论的内容
        for comment in comments:
            try:
                # 评论者名称
                username = comment.find_element(By.CSS_SELECTOR, ".f-thide").text

                # 评论内容
                content = comment.find_element(By.CSS_SELECTOR, ".cnt").text

                # 评论时间
                time_element = comment.find_element(By.CSS_SELECTOR, ".time")
                comment_time = time_element.text

                # 创建评论字典
                comment_dict = {
                    "username": username,
                    "content": content,
                    "time": comment_time
                }

                # 添加到评论列表
                all_comments.append(comment_dict)
                print(f"已收集评论: {username} - {content}")

            except Exception as e:
                print(f"收集评论时出错: {e}")

    # 浏览多个页面并收集评论
    pages_to_navigate = [
        "2", "3", "4", "5", "6",  # 直接点击的页码
        "下一页", "下一页", "下一页", "下一页"  # 点击下一页按钮
    ]

    # 收集第一页的评论
    collect_comments_on_current_page()

    # 导航到其他页面并收集评论
    for page in pages_to_navigate:
        try:
            # 点击页面链接
            link = driver.find_element(By.LINK_TEXT, page)

            # 对于"下一页"按钮，有时需要双击
            if page == "下一页":
                # 首先尝试单击
                link.click()
                time.sleep(1)

                # 如果需要双击(根据原脚本中的actions.double_click)
                if "下一页" in pages_to_navigate[pages_to_navigate.index(page)+1:]:
                    actions = ActionChains(driver)
                    actions.double_click(link).perform()
            else:
                # 普通的单击操作
                link.click()

            # 等待页面加载
            time.sleep(2)

            # 收集当前页面的评论
            collect_comments_on_current_page()

        except Exception as e:
            print(f"导航到页面 {page} 时出错: {e}")

    # 将评论保存到JSON文件
    with open("icourse_comments.json", "w", encoding="utf-8") as f:
        json.dump(all_comments, f, ensure_ascii=False, indent=4)

    print(f"成功收集了 {len(all_comments)} 条评论并保存到 icourse_comments.json")

except Exception as e:
    print(f"执行过程中出错: {e}")

finally:
    # 关闭浏览器
    driver.quit()

成功收集了 0 条评论并保存到 icourse_comments.json


In [5]:
pip install webdriver_manager

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import time
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # 创建保存页面内容的目录
    save_dir = "icourse_pages"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 访问网页
    driver.get("https://www.icourse163.org/course/NHDX-1463126169")
    driver.set_window_size(1728, 905)

    # 等待评论区加载完成
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "review-tag-num"))
    )

    # 点击评论区
    driver.find_element(By.ID, "review-tag-num").click()
    time.sleep(2)  # 等待评论区完全加载

    # 保存当前页面(第1页)评论区内容
    comment_area = driver.find_element(By.CSS_SELECTOR, ".m-review")
    comment_html = comment_area.get_attribute('outerHTML')

    # 保存第1页HTML内容
    with open(os.path.join(save_dir, "page_1.html"), "w", encoding="utf-8") as f:
        f.write(comment_html)
    print(f"已保存第1页评论区内容")

    # 页面索引计数器
    page_index = 2

    # 浏览多个页面并保存内容
    pages_to_navigate = [
        "2", "3", "4", "5", "6",  # 直接点击的页码
        "下一页", "下一页", "下一页", "下一页"  # 点击下一页按钮
    ]

    # 导航到其他页面并保存评论区内容
    for page in pages_to_navigate:
        try:
            # 点击页面链接
            link = driver.find_element(By.LINK_TEXT, page)

            # 对于"下一页"按钮，有时需要双击
            if page == "下一页":
                # 首先尝试单击
                link.click()
                time.sleep(1)

                # 如果需要双击(根据原脚本中的actions.double_click)
                if "下一页" in pages_to_navigate[pages_to_navigate.index(page)+1:]:
                    actions = ActionChains(driver)
                    actions.double_click(link).perform()
            else:
                # 普通的单击操作
                link.click()

            # 等待页面加载
            time.sleep(2)

            # 保存当前页面评论区内容
            comment_area = driver.find_element(By.CSS_SELECTOR, ".m-review")
            comment_html = comment_area.get_attribute('outerHTML')

            # 保存当前页HTML内容
            with open(os.path.join(save_dir, f"page_{page_index}.html"), "w", encoding="utf-8") as f:
                f.write(comment_html)
            print(f"已保存第{page_index}页评论区内容")

            # 递增页面索引
            page_index += 1

        except Exception as e:
            print(f"导航到页面 {page} 时出错: {e}")

    print(f"成功保存了 {page_index-1} 个页面的评论区内容到 {save_dir} 目录")

except Exception as e:
    print(f"执行过程中出错: {e}")

finally:
    # 关闭浏览器
    driver.quit()

执行过程中出错: Message: no such element: Unable to locate element: {"method":"css selector","selector":".m-review"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104beaa54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x0000000104be2cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x000000010472e864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000104775410 cxxbridge1$string$len + 382736
4   chromedriver                        0x00000001047b6480 cxxbridge1$string$len + 649088
5   chromedriver                        0x00000001047697ec cxxbridge1$string$len + 334572
6   chromedriver                        0x0000000104bafccc cxxbridge1$str$ptr + 2562928
7   chromedriver                        0x0000000104bb2f98 cxxbridge1

In [9]:
import time
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # 创建保存页面内容的目录
    save_dir = "icourse_pages"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 访问网页
    driver.get("https://www.icourse163.org/course/NHDX-1463126169")
    driver.set_window_size(1728, 905)

    # 等待评论区加载完成
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "review-tag-num"))
    )

    # 点击评论区
    driver.find_element(By.ID, "review-tag-num").click()
    time.sleep(5)  # 增加等待时间，确保评论区完全加载

    # 等待评论区出现
    try:
        comment_area = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".m-cmt"))
        )
    except:
        # 如果找不到.m-cmt，尝试其他可能的选择器
        possible_selectors = [".m-review", ".m-cmt", ".review-content", "#review-content", ".comment-list"]
        comment_area = None

        for selector in possible_selectors:
            try:
                comment_area = driver.find_element(By.CSS_SELECTOR, selector)
                print(f"找到评论区，使用选择器: {selector}")
                break
            except:
                continue

        if not comment_area:
            # 如果还是找不到，保存整个页面内容以便调试
            print("无法找到评论区，保存整个页面内容")
            comment_area = driver.find_element(By.TAG_NAME, "body")

    # 保存当前页面(第1页)评论区内容
    comment_html = comment_area.get_attribute('outerHTML')

    # 保存第1页HTML内容
    with open(os.path.join(save_dir, "page_1.html"), "w", encoding="utf-8") as f:
        f.write(comment_html)
    print(f"已保存第1页评论区内容")

    # 页面索引计数器
    page_index = 2

    # 浏览多个页面并保存内容
    pages_to_navigate = [
        "2", "3", "4", "5", "6",  # 直接点击的页码
        "下一页", "下一页", "下一页", "下一页"  # 点击下一页按钮
    ]

    # 导航到其他页面并保存评论区内容
    for page in pages_to_navigate:
        try:
            # 点击页面链接
            link = driver.find_element(By.LINK_TEXT, page)

            # 对于"下一页"按钮，有时需要双击
            if page == "下一页":
                # 首先尝试单击
                link.click()
                time.sleep(1)

                # 如果需要双击(根据原脚本中的actions.double_click)
                if "下一页" in pages_to_navigate[pages_to_navigate.index(page)+1:]:
                    actions = ActionChains(driver)
                    actions.double_click(link).perform()
            else:
                # 普通的单击操作
                link.click()

            # 等待页面加载
            time.sleep(2)

            # 保存当前页面评论区内容
            try:
                possible_selectors = [".m-review", ".m-cmt", ".review-content", "#review-content", ".comment-list"]
                comment_area = None

                for selector in possible_selectors:
                    try:
                        comment_area = driver.find_element(By.CSS_SELECTOR, selector)
                        print(f"找到评论区，使用选择器: {selector}")
                        break
                    except:
                        continue

                if not comment_area:
                    # 如果还是找不到，保存整个页面内容以便调试
                    print("无法找到评论区，保存整个页面内容")
                    comment_area = driver.find_element(By.TAG_NAME, "body")

                comment_html = comment_area.get_attribute('outerHTML')
            except Exception as e:
                print(f"获取评论区内容时出错: {e}")
                # 保存整个页面内容作为备份
                comment_html = driver.find_element(By.TAG_NAME, "body").get_attribute('outerHTML')

            # 保存当前页HTML内容
            with open(os.path.join(save_dir, f"page_{page_index}.html"), "w", encoding="utf-8") as f:
                f.write(comment_html)
            print(f"已保存第{page_index}页评论区内容")

            # 递增页面索引
            page_index += 1

        except Exception as e:
            print(f"导航到页面 {page} 时出错: {e}")

    print(f"成功保存了 {page_index-1} 个页面的评论区内容到 {save_dir} 目录")

except Exception as e:
    print(f"执行过程中出错: {e}")

finally:
    # 关闭浏览器
    driver.quit()

无法找到评论区，保存整个页面内容
已保存第1页评论区内容
无法找到评论区，保存整个页面内容
已保存第2页评论区内容
无法找到评论区，保存整个页面内容
已保存第3页评论区内容
无法找到评论区，保存整个页面内容
已保存第4页评论区内容
无法找到评论区，保存整个页面内容
已保存第5页评论区内容
无法找到评论区，保存整个页面内容
已保存第6页评论区内容
无法找到评论区，保存整个页面内容
已保存第7页评论区内容
无法找到评论区，保存整个页面内容
已保存第8页评论区内容
无法找到评论区，保存整个页面内容
已保存第9页评论区内容
无法找到评论区，保存整个页面内容
已保存第10页评论区内容
成功保存了 10 个页面的评论区内容到 icourse_pages 目录
